# Object Detection

**IoU** (Intersection over Union) is calculated as the area of overlap between two bounding boxes divided by the area of their union. 

$$ IoU = \frac{b_1 \cap b_2}{b_1 \cup b_2}, 0 \le IoU \le 1$$

While this is a good metric to evaluate object detection algorithms, it cannot be used as a loss function, because it is not differentiable when the boxes do not overlap.

To overcome this, the **GIoU** (Generalized-IoU) has been introduced

$$ GIoU = IoU - \frac{C - b_1 \cup b_2}{C} $$

It uses additionally the area of the smallest enclosing box ($C$) that contains both boxes, thus considering the relative positions of the boxes and solving the problem that non-overlapping bounding boxes have an $IoU=0$
If one box is completely inside the other, GIoU will degrade to IoU and experiments have shown that convergence can become difficult.

Next, **DIoU** (Distance-IOU)

$$ DIoU = IoU - \frac{\rho^2}{c^2} $$

**DIoU** considers the distance of the two bounding boxes ($\rho$) with respect to the enclosing box diagonal length ($c$) helping in convergence.

Finally, this has been extended to include the aspect ratio of the different bounding boxes, resulting in **CIoU** (Complete-IoU).

$$ CIoU = IoU - \frac{\rho^2}{c^2}-\alpha v $$

$\alpha v$ encapsulate the different aspect ratio of the bounding boxes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.patches as patches
import jax.numpy as jnp
import jax
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
class Evaluator:
    def __init__(self, target_box):
        self.target_box = target_box
        self.fig, self.ax = plt.subplots(1, 2, figsize=(10, 5))
        self.patch = patches.Rectangle(
            (0, 0), 0, 0, linewidth=1, edgecolor='r', facecolor='none')
        self.target_patch = patches.Rectangle(
            (target_box[0], target_box[1]), target_box[2], target_box[3], linewidth=1, edgecolor='g', facecolor='none')
        self.losses = []
        self.boxes = []

    def init(self):
        self.ax[0].set_xlim(0, 10)
        self.ax[0].set_ylim(0, 10)
        self.ax[0].add_patch(self.patch)
        self.ax[0].add_patch(self.target_patch)
        return self.patch,

    def animate(self, t):
        box = self.boxes[t]
        self.patch.set_width(box[2])
        self.patch.set_height(box[3])
        self.patch.set_xy((box[0], box[1]))
        return self.patch,

    def create_animation(self):
        self.ax[1].cla()
        self.ax[1].plot(self.losses)
        return matplotlib.animation.FuncAnimation(self.fig, func=self.animate, init_func=self.init, frames=len(self.boxes), blit=True)

    def evaluate(self, b1, metric_fn):
        def loss_fn(b1): return 1-metric_fn(b1, self.target_box)
        self.losses = []
        self.boxes = [b1]
        while metric_fn(b1, self.target_box) < 0.9:
            loss = loss_fn(b1)
            self.losses.append(loss)
            b1 -= 0.1 * jax.grad(loss_fn)(b1)
            self.boxes.append(b1)
            if len(self.losses) > 2 and self.losses[-1] == self.losses[-2]:
                print('Not converging')
                break
        return self.create_animation()


evaluator = Evaluator(jnp.array([2, 1, 5, 2], dtype=jnp.float32))

## Intersection over Union

In [ ]:
def intersection(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    xA = max(x1, x2)
    yA = max(y1, y2)
    xB = min(x1 + w1, x2 + w2)
    yB = min(y1 + h1, y2 + h2)
    return  max(0, xB - xA) * max(0, yB - yA)


def union(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    return w1 * h1 + w2 * h2 - intersection(b1, b2)


def iou(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    return intersection(b1, b2) / union(b1, b2)


b1 = jnp.array([1, 2, 5, 5], dtype=jnp.float32)
evaluator.evaluate(b1, iou)

In [ ]:
b2 = jnp.array([4, 6, 1, 1], dtype=jnp.float32)
evaluator.evaluate(b2, iou)

## Generalized Intersection over Union

In [ ]:
def enclosement(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    xA = min(x1, x2)
    yA = min(y1, y2)
    xB = max(x1 + w1, x2 + w2)
    yB = max(y1 + h1, y2 + h2)
    return (xB - xA) * (yB - yA)


def giou(b1, b2):
    R = (enclosement(b1, b2) - union(b1, b2)) / enclosement(b1, b2)
    return iou(b1, b2) - R


evaluator.evaluate(b1, giou)

In [ ]:
evaluator.evaluate(b2, giou)

## Distance Intersection over Union

In [ ]:
def distance(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    return jnp.sqrt(jnp.square(x1 - x2) + jnp.square(y1 - y2))


def enclosing_box_diagonal(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    xA = min(x1, x2)
    yA = min(y1, y2)
    xB = max(x1 + w1, x2 + w2)
    yB = max(y1 + h1, y2 + h2)
    return jnp.sqrt(jnp.square(xB - xA) + jnp.square(yB - yA))


def diou(b1, b2):
    R = jnp.square(distance(b1, b2)) / \
        jnp.square(enclosing_box_diagonal(b1, b2))
    return iou(b1, b2) - R


evaluator.evaluate(b1, diou)

In [ ]:
evaluator.evaluate(b2, diou)

## Complete Intersection over Union

In [ ]:
def ciou(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    v = 4 * jnp.square(jnp.arctan(w1 / h1) -
                       jnp.arctan(w2 / h2)) / jnp.square(jnp.pi)
    alpha = v / (1 - iou(b1, b2) + v)
    R = v * alpha
    return diou(b1, b2) - R


evaluator.evaluate(b1, ciou)

In [ ]:
evaluator.evaluate(b2, ciou)

## Sources
* [Distance-IoU Loss: Faster and Better Learning for Bounding Box Regression](https://arxiv.org/pdf/1911.08287)